In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip install JPype1==0.7.4
!pip install rhinoMorph

In [5]:
%cd /content/gdrive/My\ Drive/pytest/

/content/gdrive/My Drive/pytest


In [6]:
def read_data(filename, encoding='cp949'): # 읽기 함수 정의
    with open(filename, 'r', encoding=encoding) as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # txt 파일의 헤더(id document label)는 제외하기
    return data
def write_data(data, filename, encoding='cp949'): # 쓰기 함수 정의
    with open(filename, 'w', encoding=encoding) as f:
        f.write(data)

data = read_data('ratings_morphed.txt' , encoding='cp949')

print(len(data))
print(len(data[0])) 
print(data[0])

494
3
['8132799', '디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라 그 어렵 시절 끝 열정 지키 노라노 같 전통 있 같 사람 꿈 꾸 이루 나가 있 감사', '1']


## train data / test data split

In [10]:
data_text = [line[1] for line in data]
data_senti = [line[2] for line in data]

from sklearn.model_selection import train_test_split
# stratify: y data의 비율을 1:1에 맞게끔 조정
train_data_text, test_data_text, train_data_senti, test_data_senti = train_test_split(data_text, data_senti, stratify = data_senti)

from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq:', train_data_senti_freq)

test_data_senti_freq = Counter(test_data_senti)
print('test_data_senti_freq:', test_data_senti_freq)

train_data_senti_freq: Counter({'0': 186, '1': 184})
test_data_senti_freq: Counter({'0': 62, '1': 62})


## 행렬로 변환

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
# min_df: 최소 문자 빈도가 5이상의 단어만 대상으로
# token_pattern=r"\b\w+\b": 기본적으로 한글자는 무시하지만 한글자를 포함하려면 이 옵션 사용
vect = CountVectorizer(min_df=5).fit(train_data_text)
X_train = vect.transform(train_data_text)
print("X_train:\n", repr(X_train))

X_train:
 <370x64 sparse matrix of type '<class 'numpy.int64'>'
	with 687 stored elements in Compressed Sparse Row format>


## 행렬 내용 관찰

In [12]:
feature_names = vect.get_feature_names()
print("특성 개수:", len(feature_names))
print("처음 20개 특성:\m", feature_names[:20])

특성 개수: 64
처음 20개 특성:\m ['10점', 'ㅋㅋ', 'ㅠㅠ', 'ㅡㅡ', '감독', '감동', '그냥', '그런', '그렇', '나오', '남자', '내내', '내용', '너무', '다시', '대하', '드라마', '마음', '마지막', '만들']


## 머신러닝 알고리즘 적용

In [13]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
y_train = pd.Series(train_data_senti) # 리스트 형태를 종속변수가 될 수 있는 1차원 배열(Series)로 만든다
# solver: optimization에 사용되는 algorithm의 종류
lr = LogisticRegression(solver="liblinear") # 모델 생성
lr.fit(X_train, y_train) # 모델 훈련

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

## 테스트 데이터 입력

In [26]:
X_test = vect.transform(test_data_text)
y_test = pd.Series(test_data_senti)
# score: 예측률
# predict: 사용하면 새로운 속성들을 넣었을 때 그 레이블에 속하는지 아닌지 1또는 0으로 구성된 벡터를 반환
# predict_proba: 해당 레이블로 분류될 확률 값
print("predict:\n", lr.predict(X_test))
print("predict_proba:\n", lr.predict_proba(X_test))
print("테스트 데이터 점수:", lr.score(X_test, y_test))

predict:
 ['0' '0' '1' '0' '0' '0' '0' '0' '0' '1' '1' '1' '0' '0' '0' '1' '1' '0'
 '1' '0' '1' '1' '0' '1' '0' '0' '1' '1' '0' '1' '0' '0' '0' '1' '1' '0'
 '0' '1' '0' '0' '1' '0' '1' '0' '1' '0' '0' '1' '0' '0' '1' '1' '0' '0'
 '0' '1' '0' '1' '0' '1' '1' '0' '0' '1' '0' '0' '0' '0' '0' '0' '1' '0'
 '0' '0' '0' '0' '1' '1' '1' '1' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0'
 '0' '1' '0' '1' '0' '1' '0' '1' '0' '0' '1' '0' '0' '0' '0' '1' '1' '1'
 '0' '0' '0' '0' '0' '0' '0' '1' '0' '1' '0' '0' '0' '0' '0' '0']
predict_proba:
 [[0.51979956 0.48020044]
 [0.52804996 0.47195004]
 [0.47817323 0.52182677]
 [0.58289981 0.41710019]
 [0.8747263  0.1252737 ]
 [0.58289981 0.41710019]
 [0.6984675  0.3015325 ]
 [0.74343002 0.25656998]
 [0.69395674 0.30604326]
 [0.11101051 0.88898949]
 [0.13766549 0.86233451]
 [0.30842544 0.69157456]
 [0.52515271 0.47484729]
 [0.52804996 0.47195004]
 [0.67814066 0.32185934]
 [0.16542118 0.83457882]
 [0.13766549 0.86233451]
 [0.71873363 0.28126637]
 [0.05532724 0.94467

## 1개 데이터 예측

In [15]:
# 형태소분석기 시작
import rhinoMorph
rn = rhinoMorph.startRhino()
new_input = '오늘은 정말 재미있는 하루구나!'
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(
    rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input) # ['오늘', '정말', '재미있', '하루＇]를 한 개 문자열로 변환
inputdata.append(morphed_input) # 분석 결과를 리스트로 만들기
print('input data:', inputdata) # ['오늘 정말 재미있 하루']

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!
input data: ['오늘 정말 재미있 하루']


In [16]:
X_input = vect.transform(inputdata) # 앞에서 만든 11445 컬럼의 행렬에 적용
result = lr.predict(X_input) # 0은 부정,1은 긍정
if result == "0" : # 문자열 형태로 출력된다
    print("부정적인 글입니다")
else:
    print("긍정적인 글입니다")

긍정적인 글입니다
